In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import time

# OpenWeather API Key
api_key = "d9cc2b43f4569d1bd791efaeac304dc7"
units = "metric"

In [2]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("worldcities.csv")
cities_pd.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan


In [3]:
# Add columns for Temperature, Humidity, Cloudiness, Wind Speed
# Note that we used "" to specify initial entry.
cities_pd["Temperature_C"] = ""
cities_pd["Temperature_F"] = ""
cities_pd["Humidity"] = ""
cities_pd["Cloudiness"] = ""
cities_pd["Wind_Speed"] = ""

cities_pd.head()

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
0,Qal eh-ye Now,Qal eh-ye,34.983000,63.133300,2997.0,Afghanistan,AF,AFG,Badghis,,,,,
1,Chaghcharan,Chaghcharan,34.516701,65.250001,15000.0,Afghanistan,AF,AFG,Ghor,,,,,
2,Lashkar Gah,Lashkar Gah,31.582998,64.360000,201546.0,Afghanistan,AF,AFG,Hilmand,,,,,
3,Zaranj,Zaranj,31.112001,61.886998,49851.0,Afghanistan,AF,AFG,Nimroz,,,,,
4,Tarin Kowt,Tarin Kowt,32.633298,65.866699,10000.0,Afghanistan,AF,AFG,Uruzgan,,,,,


In [4]:
# Random selection of 500 cities with a population >1000
# Using 500 cities to stay within OpenWeatherMap limit
selected_cities = cities_pd.sample(n=10)
selected_cities = selected_cities[selected_cities["pop"].astype(int) > 1000]

# View selected_cities
selected_cities.head()
selected_cities

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
1265,Edea,Edea,3.800477,10.119992,109506.5,Cameroon,CM,CMR,Littoral,,,,,
6611,Price,Price,39.599791,-110.810017,9175.0,United States of America,US,USA,Utah,,,,,
3972,Linares,Linares,24.860380,-99.570031,52349.5,Mexico,MX,MEX,Nuevo León,,,,,
775,Molepolole,Molepolole,-24.399972,25.510008,57713.0,Botswana,BW,BWA,Kweneng,,,,,
3383,Milan,Milan,45.469975,9.205009,2125830.5,Italy,IT,ITA,Lombardia,,,,,
3476,Niigata,Niigata,37.919997,139.040030,537534.5,Japan,JP,JPN,Niigata,,,,,
2766,Nuuk,Nuuk,64.198281,-51.732659,14798.0,Greenland,GL,GRL,Kommuneqarfik Sermersooq,,,,,
1305,Terrace,Terrace,54.499992,-128.583325,14772.0,Canada,CA,CAN,British Columbia,,,,,
6723,Greenville,Greenville,34.852923,-82.394154,203256.5,United States of America,US,USA,South Carolina,,,,,


In [5]:
# Loop through the cities_pd and run a temp search for each city
for index, row in selected_cities.iterrows():
    target_url = "http://api.openweathermap.org/data/2.5/weather?" 
  
    # Wait 60 seconds before doing anything else
#     time.sleep(1)
    
    # Build query URL
    query_url = target_url + "appid=" + api_key + "&q=" + (row["city"])
#     print(query_url)
    cities_data = requests.get(query_url).json()
    try: 
        selected_cities.set_value(index, "Temperature_C", cities_data["main"]["temp"])
        selected_cities.set_value(index, "Humidity", cities_data["main"]["humidity"])
        selected_cities.set_value(index, "Cloudiness", cities_data["clouds"]["all"])
        selected_cities.set_value(index, "Wind_Speed", cities_data["wind"]["speed"])
#         print(target_url)    
                                                           
    except:
#         #
#         print("Error with data. Skipping")
        continue
                
# View selected_cities for added data
selected_cities

,city,city_ascii,lat,lng,pop,country,iso2,iso3,province,Temperature_C,Temperature_F,Humidity,Cloudiness,Wind_Speed
1265,Edea,Edea,3.800477,10.119992,109506.5,Cameroon,CM,CMR,Littoral,302.15,,74,75,2.1
6611,Price,Price,39.599791,-110.810017,9175.0,United States of America,US,USA,Utah,285.15,,43,1,0.31
3972,Linares,Linares,24.860380,-99.570031,52349.5,Mexico,MX,MEX,Nuevo León,285.332,,65,0,1.66
775,Molepolole,Molepolole,-24.399972,25.510008,57713.0,Botswana,BW,BWA,Kweneng,304.15,,15,0,4.6
3383,Milan,Milan,45.469975,9.205009,2125830.5,Italy,IT,ITA,Lombardia,300.58,,74,20,2.1
3476,Niigata,Niigata,37.919997,139.040030,537534.5,Japan,JP,JPN,Niigata,290.182,,93,92,5.66
2766,Nuuk,Nuuk,64.198281,-51.732659,14798.0,Greenland,GL,GRL,Kommuneqarfik Sermersooq,277.15,,86,75,4.6
1305,Terrace,Terrace,54.499992,-128.583325,14772.0,Canada,CA,CAN,British Columbia,279.15,,93,75,2.1
6723,Greenville,Greenville,34.852923,-82.394154,203256.5,United States of America,US,USA,South Carolina,301.72,,61,1,3.1


In [7]:
# Convert Celcius to Fahrenheit
        cel = int("Temperature_C")
        far = (9/5*(cel))+32
        selected_cities.set_value(index, "Temperature_F", far)
selected_cities.head()

IndentationError: unexpected indent (<ipython-input-7-33c8193dff92>, line 2)

In [ ]:
# Build a scatter plot for Temperature and Latitude
plt.scatter(selected_cities["Temperature_F"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Temperature vs. Latitude by City")
plt.ylabel("Temperature_F")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Temperature_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Humidity and Latitude
plt.scatter(selected_cities["Humidity"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Humidity vs. Latitude by City")
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Humidity_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Cloudiness and Latitude
plt.scatter(selected_cities["Cloudiness"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Cloudiness vs. Latitude by City")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Cloudiness_Latitude.png")

# Show plot
plt.show()

In [ ]:
# Build a scatter plot for Wind_Speed and Latitude
plt.scatter(selected_cities["Wind_Speed"], 
            selected_cities["Latitude"],
            edgecolor="black", linewidths=1, marker="o", 
            alpha=0.8, label="City")

# Incorporate the other graph properties
plt.title("Wind_Speed vs. Latitude by City")
plt.ylabel("Wind_Speed")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([-2.5, 150])
plt.ylim([-2.5, 110000])

# Save the figure
plt.savefig("output_analysis/Wind_Speed_Latitude.png")

# Show plot
plt.show()